In this notebook we'll demonstrate how to use two Canned Estimators (these encapsulate the lower-level TensorFlow code we've seen so far, and use an API loosely inspired by scikit-learn. 

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow as tf

In [17]:
(x_train, y_train), (x_test, y_test) = tf.contrib.keras.datasets.mnist.load_data()

In [18]:
print(x_train.shape)

(60000, 28, 28)


In [19]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Normalize the color values to 0-1
# (as imported, they're 0-255)
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [20]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, # repeat forever
    shuffle=True # 
)

In [21]:

test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # loop through the dataset once
    shuffle=False # don't shuffle the test data
)

In [22]:
# define the features for our model
# the names must match the input function
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

In [23]:
# LinearClassifier

estimator = tf.estimator.LinearClassifier(feature_spec, 
                                          n_classes=10,
                                          model_dir="./graphs/canned/linear")


# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/linear', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021469482C18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_i

In [24]:

# We should see about 90% accuracy here.
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-26-13:52:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-26-13:52:07
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9248, average_loss = 0.27618936, global_step = 2000, loss = 34.96068
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: ./graphs/canned/linear\model.ckpt-2000
{'accuracy': 0.9248, 'average_loss': 0.27618936, 'loss': 34.96068, 'global_step': 2000}


In [25]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


In [27]:

estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/deep', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002145C6976D8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2000 into ./graphs/canned/deep\model.ckpt.
INFO:tensorflow:loss = 7.711816, step = 2001
INFO:tensorflow:global_step/sec: 174.14
INFO:tensorflow:loss = 7.8134885, step = 2101 (0.579 sec)
INFO:tensorflow:global_step/sec: 217.029
INFO:tensorflow:loss = 7.8732204, step = 2201 (0.459 sec)
INFO:tensorflow:global_step/sec: 194.542
INFO:tensorflow:loss = 4.5014606, step = 2301 (0.516 sec)
INFO:tensorflow:global_step/sec: 191.332
INFO:tensorflow:loss = 3.662867, step = 2401 (0.522 sec)
INFO:tensorflow:global_step/sec: 207.592
INFO:tensorflow:loss = 1.9076159, step = 2501 (0.481 sec)
INFO:tensorflow:global_step/sec: 202.085
INFO:tensorflow:loss = 

In [29]:

# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-26-13:52:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-26-13:52:25
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.9777, average_loss = 0.07061621, global_step = 4000, loss = 8.938761
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: ./graphs/canned/deep\model.ckpt-4000
{'accuracy': 0.9777, 'average_loss': 0.07061621, 'loss': 8.938761, 'global_step': 4000}


In [30]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


In [31]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/deep', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002146937A898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:

# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4000 into ./graphs/canned/deep\model.ckpt.
INFO:tensorflow:loss = 1.3653553, step = 4001
INFO:tensorflow:global_step/sec: 174.988
INFO:tensorflow:loss = 1.4647474, step = 4101 (0.575 sec)
INFO:tensorflow:global_step/sec: 197.2
INFO:tensorflow:loss = 0.6456722, step = 4201 (0.506 sec)
INFO:tensorflow:global_step/sec: 205.778
INFO:tensorflow:loss = 4.871215, step = 4301 (0.487 sec)
INFO:tensorflow:global_step/sec: 194.316
INFO:tensorflow:loss = 2.4340227, step = 4401 (0.515 sec)
INFO:tensorflow:global_step/sec: 183.87
INFO:tensorflow:loss = 3.2014341, step = 4501 (0.544 sec)
INFO:tensorflow:global_step/sec: 192.388
INFO:tensorflow:loss = 1

In [33]:
# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-26-13:54:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-26-13:54:48
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.9777, average_loss = 0.07252213, global_step = 6000, loss = 9.1800165
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6000: ./graphs/canned/deep\model.ckpt-6000
{'accuracy': 0.9777, 'average_loss': 0.07252213, 'loss': 9.1800165, 'global_step': 6000}
